### Challenge 1
* Open up a new IPython notebook
* Download a few MTA turnstile data files
* Open up a file, use csv reader to read it, make a python dict where there is a key for each (C/A, UNIT, SCP, STATION). These are the first four columns. The value for this key should be a list of lists. Each list in the list is the rest of the columns in a row. For example, one key-value pair should look like

{ ('A002','R051','02-00-00','LEXINGTON AVE'): [ ['NQR456', 'BMT', '01/03/2015', '03:00:00', 'REGULAR', '0004945474', '0001675324'], ['NQR456', 'BMT', '01/03/2015', '07:00:00', 'REGULAR', '0004945478', '0001675333'], ['NQR456', 'BMT', '01/03/2015', '11:00:00', 'REGULAR', '0004945515', '0001675364'], ... ] }

In [167]:
import pandas as pd
import numpy as np
import itertools
import datetime

In [168]:
# Load data
path = 'turnstile_160402.txt'
df = pd.read_csv(path, index_col=None)
df = df.rename(columns=lambda x: x.rstrip())
df = df.sort_values(['C/A', 'UNIT', 'SCP', 'STATION'], ascending=[True, True, True, True])
df.head(3)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,00:00:00,REGULAR,5595746,1893277
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,04:00:00,REGULAR,5595746,1893282
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,08:00:00,REGULAR,5595746,1893282


#### Challenge 2

Let's turn this into a time series.

For each key (basically the control area, unit, device address and
 station of a specific turnstile), have a list again, but let the list
 be comprised of just the point in time and the count of entries.

This basically means keeping only the date, time, and entries fields
in each list. You can convert the date and time into datetime objects
-- That is a python class that represents a point in time. You can
combine the date and time fields into a string and use the
[dateutil](https://labix.org/python-dateutil) module to convert it
into a datetime object. For an example check
[this StackOverflow question](http://stackoverflow.com/questions/23385003/attributeerror-when-using-import-dateutil-and-dateutil-parser-parse-but-no).

Your new dict should look something like

{    ('A002','R051','02-00-00','LEXINGTON AVE'):
[
[datetime.datetime(2013, 3, 2, 3, 0), 3788],
[datetime.datetime(2013, 3, 2, 7, 0), 2585],
[datetime.datetime(2013, 3, 2, 12, 0), 10653],
[datetime.datetime(2013, 3, 2, 17, 0), 11016],
[datetime.datetime(2013, 3, 2, 23, 0), 10666],
[datetime.datetime(2013, 3, 3, 3, 0), 10814],
[datetime.datetime(2013, 3, 3, 7, 0), 10229],
...
],
....
}

In [169]:
df['DATETIME'] = df['DATE'] + ' ' + df['TIME']
df = df.sort_values(['C/A', 'UNIT', 'SCP', 'STATION', 'DATETIME'], ascending=[True, True, True, True, True])
df['DATETIME'] = pd.to_datetime(df['DATETIME'], format='%m/%d/%Y %X')
df.head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
0,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,00:00:00,REGULAR,5595746,1893277,2016-03-26 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,04:00:00,REGULAR,5595746,1893282,2016-03-26 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,08:00:00,REGULAR,5595746,1893282,2016-03-26 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,12:00:00,REGULAR,5595746,1893282,2016-03-26 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,16:00:00,REGULAR,5595746,1893282,2016-03-26 16:00:00
5,A002,R051,02-00-00,59 ST,NQR456,BMT,03/26/2016,20:00:00,REGULAR,5595746,1893282,2016-03-26 20:00:00
6,A002,R051,02-00-00,59 ST,NQR456,BMT,03/27/2016,00:00:00,REGULAR,5595746,1893282,2016-03-27 00:00:00
7,A002,R051,02-00-00,59 ST,NQR456,BMT,03/27/2016,04:00:00,REGULAR,5595746,1893282,2016-03-27 04:00:00
8,A002,R051,02-00-00,59 ST,NQR456,BMT,03/27/2016,08:00:00,REGULAR,5595746,1893282,2016-03-27 08:00:00
9,A002,R051,02-00-00,59 ST,NQR456,BMT,03/27/2016,12:00:00,REGULAR,5595746,1893282,2016-03-27 12:00:00


In [170]:
df = df[['C/A', 'UNIT', 'SCP', 'STATION', 'DATE', 'DATETIME', 'ENTRIES', 'EXITS']]

# Take diff between each row
df['ENTRIES_DIFF'] = df['ENTRIES'].diff()
df['EXITS_DIFF'] = df['EXITS'].diff()

# Define filter equations
def filter_not_equal(df, df_col, criterion):
    the_filter = df[df_col] != criterion
    df = df[the_filter]
    return df

def filter_get_atleast(df, df_col, criterion):
    the_filter = df[df_col] >= criterion
    df = df[the_filter]
    return df

def filter_get_lessthan(df, df_col, criterion):
    the_filter = df[df_col] < criterion
    df = df[the_filter]
    return df


# Filter out NaN's and any negative values
df = filter_not_equal(df, 'DATETIME', pd.to_datetime('2016-3-26 00:00:00'))
df = filter_get_atleast(df, 'ENTRIES_DIFF', 0)
df = filter_get_atleast(df, 'EXITS_DIFF', 0)
df = filter_get_lessthan(df, 'ENTRIES_DIFF', 100000)
df = filter_get_lessthan(df, 'EXITS_DIFF', 100000)

df.head(100)

,C/A,UNIT,SCP,STATION,DATE,DATETIME,ENTRIES,EXITS,ENTRIES_DIFF,EXITS_DIFF
1,A002,R051,02-00-00,59 ST,03/26/2016,2016-03-26 04:00:00,5595746,1893282,0.0,5.0
2,A002,R051,02-00-00,59 ST,03/26/2016,2016-03-26 08:00:00,5595746,1893282,0.0,0.0
3,A002,R051,02-00-00,59 ST,03/26/2016,2016-03-26 12:00:00,5595746,1893282,0.0,0.0
4,A002,R051,02-00-00,59 ST,03/26/2016,2016-03-26 16:00:00,5595746,1893282,0.0,0.0
5,A002,R051,02-00-00,59 ST,03/26/2016,2016-03-26 20:00:00,5595746,1893282,0.0,0.0
6,A002,R051,02-00-00,59 ST,03/27/2016,2016-03-27 00:00:00,5595746,1893282,0.0,0.0
7,A002,R051,02-00-00,59 ST,03/27/2016,2016-03-27 04:00:00,5595746,1893282,0.0,0.0
8,A002,R051,02-00-00,59 ST,03/27/2016,2016-03-27 08:00:00,5595746,1893282,0.0,0.0
9,A002,R051,02-00-00,59 ST,03/27/2016,2016-03-27 12:00:00,5595746,1893282,0.0,0.0
10,A002,R051,02-00-00,59 ST,03/27/2016,2016-03-27 16:00:00,5595746,1893282,0.0,0.0


#### Challenge 3

These counts are for every n hours. (What is n?) We want total daily entries.

Now make it that we again have the same keys, but now we have a single
value for a single day, which is the total number of passengers that
entered through this turnstile on this day.

In [171]:
df_daily = df[['C/A', 'UNIT', 'SCP', 'STATION', 'DATE', 'DATETIME', 'ENTRIES_DIFF', 'EXITS_DIFF']]
df_daily = df_daily.groupby(['C/A', 'UNIT','SCP','STATION','DATE']).sum()
df_daily['TOTAL_DIFF'] = df_daily['ENTRIES_DIFF'] + df_daily['EXITS_DIFF']
df_daily.head(100)

ENTRIES_DIFF  EXITS_DIFF  TOTAL_DIFF
C/A  UNIT SCP      STATION    DATE                                            
A002 R051 02-00-00 59 ST      03/26/2016           0.0         5.0         5.0
                              03/27/2016           0.0         0.0         0.0
                              03/28/2016         945.0       652.0      1597.0
                              03/29/2016        1144.0       875.0      2019.0
                              03/30/2016        1144.0       848.0      1992.0
                              03/31/2016         454.0       318.0       772.0
                              04/01/2016        1207.0       284.0      1491.0
          02-00-01 59 ST      03/26/2016           0.0         6.0         6.0
                              03/27/2016           0.0         0.0         0.0
                              03/28/2016         875.0       457.0      1332.0
                              03/29/2016         981.0       657.0      1638.0
                              03/30/2016        1057.0       558.0      1615.0
                              03/31/2016         410.0       183.0       593.0
                              04/01/2016        1047.0       167.0      1214.0
          02-03-00 59 ST      03/26/2016           0.0        17.0        17.0
                              03/27/2016           0.0         0.0         0.0
                              03/28/2016         506.0      2532.0      3038.0
                              03/29/2016         782.0      2710.0      3492.0
                              03/30/2016         695.0      2885.0      3580.0
                              03/31/2016         864.0      3913.0      4777.0
                              04/01/2016         641.0      3392.0      4033.0
          02-03-01 59 ST      03/26/2016           0.0        15.0        15.0
                              03/27/2016           0.0         0.0         0.0
                              03/28/2016        1212.0      2308.0      3520.0
                              03/29/2016        1654.0      2476.0      4130.0
                              03/30/2016        1736.0      2535.0      4271.0
                              03/31/2016        1924.0      2985.0      4909.0
                              04/01/2016        1624.0      2771.0      4395.0
          02-03-02 59 ST      03/26/2016          18.0         1.0        19.0
                              03/27/2016           0.0         0.0         0.0
...                                                ...         ...         ...
          02-05-01 59 ST      03/26/2016           0.0         0.0         0.0
                              03/27/2016           0.0         6.0         6.0
                              03/28/2016           0.0         0.0         0.0
                              03/29/2016           0.0         0.0         0.0
                              03/30/2016           0.0         0.0         0.0
                              03/31/2016           0.0         0.0         0.0
                              04/01/2016           0.0         0.0         0.0
          02-06-00 59 ST      03/26/2016         726.0        25.0       751.0
                              03/27/2016         275.0        26.0       301.0
                              03/28/2016        1185.0        15.0      1200.0
                              03/29/2016        1386.0        24.0      1410.0
                              03/30/2016        1339.0        15.0      1354.0
                              03/31/2016        1433.0        28.0      1461.0
                              04/01/2016        2003.0        17.0      2020.0
A006 R079 00-00-00 5 AV/59 ST 03/26/2016           0.0         2.0         2.0
                              03/27/2016           0.0         4.0         4.0
                              03/28/2016        1671.0      2609.0      4280.0
                              03/29/2016        1859.0      3034.0      4893.0
             

In [172]:
import pickle

df_daily.to_pickle('df_daily.pickle')